It might be useful to run 'pip3 install &lt;library&gt; --upgrade" to make sure you have latest versions of pandas, matplotlib, etc.

In [20]:
import gzip            
import io              
import numpy as np
import pandas as pd    
import pymysql.cursors 
import rdflib
from rdflib import Namespace
import urllib.request  

import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [21]:
# Connect to the database
# I will email you the file sebastia_adsq_params.txt . It's just a four line text file.
# Put that in directory and the following line will read it.
db_params = open("sebastia_adsqro_params.txt").read().split()

In [3]:
# Connect to the database

connection = pymysql.connect(host=db_params[0],
                             user=db_params[1],
                             password=db_params[2],
                             db=db_params[3],
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

## Turn 'ramphs' into Triples

In [22]:
sql = "select * from ramphs"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)
df.head()

,id,title,label,pleiades,welch,golvin,type,capacity,modcountry,province,...,certainty,youtube,extmajor,extminor,arenamajor,arenaminor,latitude,longitude,elevation,pleiades_id
0,duraEuroposAmphitheater,Amphitheater at Dura Europos,Dura,http://pleiades.stoa.org/places/893989,,129,amphitheater,1000.0,Syria,syria,...,,,50.0,44.0,31.0,25.0,34.74985500000,40.7289260000,223,893989
1,arlesAmphitheater,Amphitheater at Arles,Arles,http://pleiades.stoa.org/places/148217,,154,amphitheater,20000.0,France,narbonensis,...,,https://www.youtube.com/watch?v=oCz-76hb1LU,136.0,107.0,47.0,32.0,43.67777800000,4.6311110000,21,148217
2,lyonAmphitheater,Amphitheater at Lyon,Lyon,http://pleiades.stoa.org/places/167717,,,amphitheater,20000.0,France,lugdunensis,...,,,105.0,NaN,67.6,42.0,45.77055600000,4.8305560000,206,167717
3,ludusMagnusArena,Ludus Magnus Arena,Ludus Magnus,http://pleiades.stoa.org/places/423025,,,practice-arena,NaN,Italy,,...,,,NaN,NaN,NaN,NaN,41.88995000000,12.4949130000,22,423025
4,romeFlavianAmphitheater,Flavian Amphitheater at Rome,Colosseum,http://pleiades.stoa.org/places/423025,,152,amphitheater,50000.0,Italy,,...,,https://www.youtube.com/watch?v=kxwenPo1grc,189.0,156.0,83.0,48.0,41.89016900000,12.4922690000,22,423025


In [23]:
pseudo_triples = pd.melt(df,id_vars='id')
pseudo_triples.replace('', np.nan, inplace=True)
pseudo_triples.dropna(inplace=True)
# pseudo_triples.head()

In [24]:
g = rdflib.Graph()
RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
EXclass = Namespace("http://example.org/class/")
EXid = Namespace("http://example.org/id/")
EXproperty = Namespace("http://example.org/property/")

In [25]:
for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['id'])
    p = rdflib.URIRef('http://example.org/property/%s' % t[1]['variable'])

    
    g.add((s,rdflib.URIRef(RDF.type),EXclass.amphitheater))
    if t[1]['variable'] in ['arenamajor','arenaminor','capacity','elevation','extmajor','extminor','longitude','latitude']:
        o = rdflib.Literal(float(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['pleiades']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [26]:
result = g.query(
        """SELECT * WHERE {
            ?s <http://example.org/property/capacity> ?o .
                } """)
pd.DataFrame(result.bindings)

,o,s
0,2309.0,http://example.org/id/roselleAmphitheater
1,4398.0,http://example.org/id/tipasaAmphitheater
2,15550.0,http://example.org/id/octodurusForumClaudiiVal...
3,15000.0,http://example.org/id/meridaAmphitheater
4,26000.0,http://example.org/id/pulaAmphitheater
5,11600.0,http://example.org/id/botriaAmphitheater
6,12000.0,http://example.org/id/bararusAmphitheater
7,2729.0,http://example.org/id/flaviaSolvaAmphitheater
8,7200.0,http://example.org/id/seressiAmphitheater
9,17000.0,http://example.org/id/grandHalfAmphitheater


## Turn 'chronogroups' into Triples

In [27]:
sql = "select * from ramphs_chronogrps"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)
df.head()

,id,start,end
0,Republican,-70,-31
1,Caesarean,-49,-44
2,Augustan,-27,14
3,Julio-Claudian,-27,68
4,Neronian,54,68


In [28]:
df.rename(index=str, columns={"id": "label"}, inplace = True)
newcol = df['label'].replace(r' ', '', regex = True)
df = df.assign(idd = newcol)
df.rename(index=str, columns={"idd": "id"}, inplace = True)
df

,label,start,end,id
0,Republican,-70,-31,Republican
1,Caesarean,-49,-44,Caesarean
2,Augustan,-27,14,Augustan
3,Julio-Claudian,-27,68,Julio-Claudian
4,Neronian,54,68,Neronian
5,Flavian,69,96,Flavian
6,First Century,1,99,FirstCentury
7,Late1stEarly2nd,75,125,Late1stEarly2nd
8,Hadrianic,117,138,Hadrianic
9,Second Century,100,199,SecondCentury


In [29]:
pseudo_triples = pd.melt(df,id_vars='id').sort_values('id')
pseudo_triples.replace('', np.nan, inplace=True)
pseudo_triples.dropna(inplace=True)

for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['id'])
    p = rdflib.URIRef('http://example.org/property/%s' % t[1]['variable'])
    
    g.add((s,RDF.type,EXclass.chronogrp))
    if t[1]['variable'] in ['start','end']:
        o = rdflib.Literal(float(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [30]:
result = g.query(
        """SELECT ?p ?o WHERE {
            <http://example.org/id/FirstCentury> ?p ?o .
                } LIMIT 10""")

In [31]:
# or
pd.DataFrame(result.bindings)

,o,p
0,First Century,http://example.org/property/label
1,1.0,http://example.org/property/start
2,http://example.org/class/chronogrp,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
3,99.0,http://example.org/property/end


In [32]:
result = g.query(
        """SELECT * WHERE {
            ?ramph a <http://example.org/class/amphitheater> ;
                    <http://example.org/property/chronogrp> ?rchrono .
                    
            ?chronogrp <http://example.org/property/label> ?rchrono ;
                       <http://example.org/property/end> ?end
                } ORDER BY ?end""")

pd.DataFrame(result.bindings)

,chronogrp,end,ramph,rchrono
0,http://example.org/id/Caesarean,-44.0,http://example.org/id/corinthAmphitheater,Caesarean
1,http://example.org/id/Caesarean,-44.0,http://example.org/id/carmonaAmphitheater,Caesarean
2,http://example.org/id/Caesarean,-44.0,http://example.org/id/espejoAmphitheater,Caesarean
3,http://example.org/id/Republican,-31.0,http://example.org/id/teanoAmphitheater,Republican
4,http://example.org/id/Republican,-31.0,http://example.org/id/pozzuoliEarlyAmphitheater,Republican
5,http://example.org/id/Republican,-31.0,http://example.org/id/liternoAmphitheater,Republican
6,http://example.org/id/Republican,-31.0,http://example.org/id/cumaeAmphitheater,Republican
7,http://example.org/id/Republican,-31.0,http://example.org/id/beneventoAmphitheater,Republican
8,http://example.org/id/Republican,-31.0,http://example.org/id/paestumAmphitheater,Republican
9,http://example.org/id/Republican,-31.0,http://example.org/id/pompeiiAmphitheater,Republican


In [33]:
g.serialize(destination='ramphs.ttl', format='turtle')